In [2]:
import sys
import json
import math

import urllib.request
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from datetime import datetime

### Download FEMA data using their API

In [2]:
# Adapted from https://www.fema.gov/about/openfema/developer-resources

#!/usr/bin/env python3
# Paging example using Python 3. Output in JSON.

def fema_api_download(baseUrl, fileName):
    top = 1000      # number of records to get per call
    skip = 0        # number of records to skip

    # Return 1 record with your criteria to get total record count. 
    # Specifying only 1 column here to reduce amount of data returned. 
    # Need inlinecount to get record count. 
    webUrl = urllib.request.urlopen(baseUrl + '&$inlinecount=allpages&$select=id&$top=1')
    result = webUrl.read()
    jsonData = json.loads(result.decode())

    # Calculate the number of calls we will need to get all of our data (using the maximum of 1000)
    recCount = jsonData['metadata']['count']
    loopNum = math.ceil(recCount / top)

    # Logging
    print('Starting download @ {}, {} records, {} records per call, {} iterations needed'.format(
          str(datetime.now()), str(recCount), str(top), str(loopNum)))

    # Initialize the output file. 
    outFile = open(fileName, 'w')
    outFile.write('{"fema_open_api":[')

    # Loop and call the API endpoint changing the record start each iteration. The metadata is being
    #   suppressed as we no longer need it.
    i = 0
    while (i < loopNum):
        # By default data is returned as a JSON object, the data set name being the root element. Unless
        #   you extract records as you process, you will end up with 1 distinct JSON object for EVERY 
        #   call/iteration. An alternative is to return the data as JSONA (an array of json objects) with 
        #   no root element - just a bracket at the start and end. This is easier to manipulate.
        webUrl = urllib.request.urlopen(baseUrl + "&$metadata=off&$format=jsona&$skip=" + str(skip) + "&$top=" + str(top))
        result = webUrl.read()

        # The data is already returned in a JSON format. There is no need to decode and load as a JSON object.
        #   If you want to begin working with and manipulating the JSON, import the json library and load with
        #   something like: jsonData = json.loads(result.decode())

        # Append results to file, trimming off first and last JSONA brackets, adding comma except for last call,
        #   AND root element terminating array bracket and brace to end unless on last call. The goal here is to 
        #   create a valid JSON file that contains ALL the records. This can be done differently.
        if (i == (loopNum - 1)):
            # on the last so terminate the single JSON object
            outFile.write(str(result[1:-1],'utf-8') + "]}")
        else:
            outFile.write(str(result[1:-1],'utf-8') + ",")

        # increment the loop counter and skip value
        i += 1
        skip = i * top

        print("Iteration " + str(i) + " done")

    print('Data downloaded to {}'.format(fileName))
    outFile.close()

In [ ]:
# BaseUrl for Hurricane Maria Disaster (4339) in PR
baseUrl = 'https://www.fema.gov/api/open/v1/IndividualsAndHouseholdsProgramValidRegistrations?$filter=disasterNumber%20eq%204339%20and%20damagedStateAbbreviation%20eq%20%27PR%27'

# BaseUrl for Hurricane Maria Disaster (4339) across all states
#baseUrl = 'https://www.fema.gov/api/open/v1/IndividualsAndHouseholdsProgramValidRegistrations?$filter=disasterNumber%20eq%204339'

# BaseUrl for Disaster Summaries in PR
#baseUrl = 'https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries?$filter=state%20eq%20%27PR%27'

fema_api_download(baseUrl=baseUrl, fileName='../data/FEMA-DR-4339-PR.json')

In [3]:
# Check the number of records downloaded
femaFile = open('../data/FEMA-DR-4339-PR.json', 'r')
femaData = json.load(femaFile)
print('Found {} records in file'.format(str(len(femaData['fema_open_api']))))

# Convert to Pandas DataFrame
femaDf = json_normalize(femaData['fema_open_api'])
femaFile.close()

Found 1120767 records in file


In [4]:
# Look at few lines
femaDf.head()

,accessFunctionalNeeds,applicantAge,autoDamage,county,damagedCity,damagedStateAbbreviation,damagedZipCode,declarationDate,destroyed,disasterNumber,...,roofDamage,roofDamageAmount,rpfvl,sbaApproved,sbaEligible,shelterNeed,tsaCheckedIn,tsaEligible,utilitiesOut,waterLevel
0,False,50-64,False,Caguas (Municipio),CAGUAS,PR,00725,2017-09-21T01:00:00.000Z,False,4339,...,False,0.00,0.00,False,False,None,False,True,True,0
1,False,35-49,False,Ponce (Municipio),PONCE,PR,00728,2017-09-21T01:00:00.000Z,False,4339,...,True,1731.60,3792.06,False,False,None,False,True,True,0
2,False,35-49,False,Trujillo Alto (Municipio),TRUJILLO ALTO,PR,00976,2017-09-21T01:00:00.000Z,False,4339,...,True,958.28,4327.35,True,True,True,False,True,True,0
3,True,35-49,False,Humacao (Municipio),PUNTA SANTIAGO,PR,00741,2017-09-21T01:00:00.000Z,False,4339,...,False,0.00,1108.41,False,False,None,False,True,True,2
4,False,50-64,False,Catano (Municipio),CATANO,PR,00962,2017-09-21T01:00:00.000Z,False,4339,...,False,0.00,2281.52,False,False,None,False,True,True,0


In [ ]:
# Convert to CSV
femaDf.to_csv('summaries.csv', index=False, encoding='utf-8')

### Aggregate 'haStatus' column across all counties - Top 20 reasons

In [5]:
femaDf.groupby(['haStatus'])['haStatus'].count().nlargest(30)

haStatus
TSA - Transitional Sheltering Assistance                                                                      799248
INR- Ineligible - No Relocation                                                                                59423
ER - Eligible - Rental Assistance                                                                              52472
IID- Ineligible - Insufficient Damage                                                                          38229
EHR - Eligible - Home Repairs                                                                                  36161
IID,IOWNV - Ineligible Insufficient Damage, Ownership Not Verified                                              7935
INO - Ineligible - Other                                                                                        6565
IID,ILDOBR - Ineligible Insufficient Damage, Ineligible Linked For Duplicate Review                             6474
WVO - Withdrawn - Applicant Withdrew Voluntarily       

### Aggregate 'inspnReturned' column, grouped by county - Top 20 counties with lowest rates of inspection

In [6]:
femaDf.groupby(['county'])['inspnReturned'].mean().nsmallest(20)*100

county
Statewide                     0.000000
Sabana Grande (Municipio)    42.470192
Cabo Rojo (Municipio)        44.457605
San German (Municipio)       45.859193
Hormigueros (Municipio)      46.990064
Lajas (Municipio)            50.863091
Mayaguez (Municipio)         51.601079
Yauco (Municipio)            55.542187
Ponce (Municipio)            59.781048
Camuy (Municipio)            60.652921
San Sebastian (Municipio)    60.727304
San Juan (Municipio)         60.924678
Lares (Municipio)            61.057405
Hatillo (Municipio)          61.575137
Guaynabo (Municipio)         61.731992
Guanica (Municipio)          61.930157
Moca (Municipio)             62.308241
Bayamon (Municipio)          62.335844
Guayanilla (Municipio)       62.444136
Aguadilla (Municipio)        62.717552
Name: inspnReturned, dtype: float64

### Aggregate 'ihpAmount' column, grouped by county - Top 20 counties

In [7]:
# View aggregated ihpAmount, grouped by county
femaDf.groupby(['county'])['ihpAmount'].agg('sum').nlargest(20)

county
San Juan (Municipio)        6.658102e+07
Toa Baja (Municipio)        5.422226e+07
Ponce (Municipio)           5.157847e+07
Bayamon (Municipio)         4.904513e+07
Arecibo (Municipio)         4.104821e+07
Caguas (Municipio)          3.908064e+07
Canovanas (Municipio)       3.444231e+07
Humacao (Municipio)         3.320571e+07
Carolina (Municipio)        2.987918e+07
Toa Alta (Municipio)        2.841064e+07
Vega Baja (Municipio)       2.750502e+07
Guayama (Municipio)         2.579062e+07
Corozal (Municipio)         2.567433e+07
Yabucoa (Municipio)         2.481859e+07
Naranjito (Municipio)       2.342492e+07
Salinas (Municipio)         2.328003e+07
Morovis (Municipio)         2.297588e+07
Juana Diaz (Municipio)      2.260435e+07
Guaynabo (Municipio)        2.209636e+07
Barranquitas (Municipio)    2.161582e+07
Name: ihpAmount, dtype: float64

### Aggregate 'roofDamage' column, grouped by county - Top 20 counties with most roof damage

In [8]:
# View aggregated ihpAmount, grouped by county
femaDf.groupby(['county'])['roofDamage'].mean().nlargest(20)*100

county
Adjuntas (Municipio)      25.021842
Salinas (Municipio)       24.773889
Orocovis (Municipio)      23.278939
Aibonito (Municipio)      21.607554
Maunabo (Municipio)       21.533923
Comerio (Municipio)       21.424252
Morovis (Municipio)       21.348315
Naranjito (Municipio)     21.158587
Ciales (Municipio)        20.771124
Maricao (Municipio)       20.310633
Villalba (Municipio)      20.278279
Utuado (Municipio)        19.946730
Penuelas (Municipio)      19.854083
Loiza (Municipio)         19.846234
Yabucoa (Municipio)       19.758728
Patillas (Municipio)      19.756338
Corozal (Municipio)       19.515001
Guayama (Municipio)       19.407384
Juana Diaz (Municipio)    19.144629
Arroyo (Municipio)        18.845444
Name: roofDamage, dtype: float64

### Aggregate registrations by county

In [41]:
femaDf['county'].value_counts()

San Juan (Municipio)         122594
Bayamon (Municipio)           63583
Carolina (Municipio)          52319
Ponce (Municipio)             50148
Caguas (Municipio)            42829
Arecibo (Municipio)           30047
Guaynabo (Municipio)          28141
Toa Baja (Municipio)          27799
Mayaguez (Municipio)          24827
Toa Alta (Municipio)          21041
Trujillo Alto (Municipio)     20902
Humacao (Municipio)           18984
Vega Baja (Municipio)         18952
Aguadilla (Municipio)         17582
Rio Grande (Municipio)        16012
Canovanas (Municipio)         15967
Juana Diaz (Municipio)        15806
Cayey (Municipio)             14581
Cabo Rojo (Municipio)         14353
Manati (Municipio)            13735
Cidra (Municipio)             13416
Yauco (Municipio)             13298
Juncos (Municipio)            13203
Gurabo (Municipio)            12907
Guayama (Municipio)           12892
Isabela (Municipio)           12723
Aguada (Municipio)            12666
Coamo (Municipio)           

### Some stats around inspnReturned, homeOwnersInsurance, floodInsurance, ihpEligible, inspnIssued

In [14]:
femaDf['inspnReturned'].value_counts()

True     757044
False    363723
Name: inspnReturned, dtype: int64

In [15]:
femaDf['homeOwnersInsurance'].value_counts()

False    951104
True     169663
Name: homeOwnersInsurance, dtype: int64

In [16]:
femaDf['floodInsurance'].value_counts()

False    1086575
True       34192
Name: floodInsurance, dtype: int64

In [320]:
femaDf['homeDamage'].mean()

0.6447307372470922

In [326]:
femaDf['homeDamage'].value_counts()

True     722591
False    398173
Name: homeDamage, dtype: int64

In [322]:
femaDf['utilitiesOut'].mean()

0.9430487959408402

In [327]:
femaDf['homeDamage'].mean()

0.6447307372470922

In [313]:
len(femaDf[(femaDf['ihpAmount'] == 0) & (femaDf['homeDamage'])])

379337

### Importing PR county data (../data/county-demographics.csv)

In [8]:
import unidecode

# Load county data scraped from Wikipedia
countyDemographicsDf = pd.read_csv('../data/county-demographics.csv', delimiter=',')

# Format the county column
countyDemographicsDf['county_name'] = countyDemographicsDf.apply(lambda row: unidecode.unidecode(row['county_name']), axis=1)
countyDemographicsDf['county_name'] = countyDemographicsDf.apply(lambda row: row['county_name'].split(' Municipio')[0], axis=1)

countyDemographicsDf.head(20)

,fips_code,county_name,pop_estimates_2019,pop_estimates_2010,pop_percent_change,pop_census_2010,pop_under_5yrs_percent,pop_under_18yrs_percent,pop_65yrs_over_percent,female_percent,...,nonemployer_establishments,all_firms,male_owned_firms,female_owned_firms,minority_owned_firms,nonminority_owned_firms,veteran_owned_firms,nonveteran_owned_firms,pop_per_square_mile,land_area_square_miles
0,72001,Adjuntas,17363,19483,-0.109,19483,0.037,0.190,0.204,0.512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.1,66.69
1,72003,Aguada,36694,41959,-0.125,41959,0.034,0.167,0.202,0.511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1360.1,30.85
2,72005,Aguadilla,50265,60949,-0.175,60949,0.036,0.182,0.225,0.517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1668.5,36.53
3,72007,Aguas Buenas,24814,28662,-0.134,28662,0.033,0.179,0.208,0.517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,952.6,30.09
4,72009,Aibonito,22108,25900,-0.146,25900,0.039,0.178,0.219,0.521,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,827.2,31.31
5,72011,Anasco,26161,29261,-0.106,29261,0.031,0.168,0.213,0.517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,744.8,39.29
6,72013,Arecibo,81966,96440,-0.150,96440,0.037,0.175,0.224,0.524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,765.7,125.95
7,72015,Arroyo,17238,19575,-0.119,19575,0.037,0.202,0.204,0.539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1304.3,15.01
8,72017,Barceloneta,23727,24816,-0.044,24816,0.039,0.185,0.201,0.527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1327.6,18.69
9,72019,Barranquitas,27725,30318,-0.086,30318,0.048,0.211,0.168,0.507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,885.1,34.25


### Merge the data sources

In [12]:
# Extract the aggregated ihpAmounts per county
ihpAndIncomePerCapitaDf = femaDf.groupby(['county'])['ihpAmount'].agg('sum').reset_index()

# Drop the Statewide row
ihpAndIncomePerCapitaDf = ihpAndIncomePerCapitaDf[ihpAndIncomePerCapitaDf['county'] != 'Statewide']

# Format the county column
ihpAndIncomePerCapitaDf['county'] = ihpAndIncomePerCapitaDf.county.str.extract('(.+?) \(')

ihpAndIncomePerCapitaDf = pd.merge(ihpAndIncomePerCapitaDf, 
                                   countyDemographicsDf[['county_name','pop_estimates_2019', 'per_capita_income_past_year', 'fips_code']],
                                   left_on='county', right_on='county_name', how='inner')

# Calculate the ihpAmount per capita
ihpAndIncomePerCapitaDf['ihpAmountPerCapita'] = np.round(ihpAndIncomePerCapitaDf['ihpAmount']/ihpAndIncomePerCapitaDf['pop_estimates_2019']*100,decimals=2)

ihpAndIncomePerCapitaDf.head(20)

,county,ihpAmount,county_name,pop_estimates_2019,per_capita_income_past_year,fips_code,ihpAmountPerCapita
0,Adjuntas,1.200609e+07,Adjuntas,17363,7595,72001,69147.57
1,Aguada,1.851387e+07,Aguada,36694,9692,72003,50454.77
2,Aguadilla,1.623414e+07,Aguadilla,50265,11298,72005,32297.11
3,Aguas Buenas,1.255817e+07,Aguas Buenas,24814,11097,72007,50609.20
4,Aibonito,1.732764e+07,Aibonito,22108,11363,72009,78377.23
5,Anasco,1.703423e+07,Anasco,26161,10603,72011,65113.09
6,Arecibo,4.104821e+07,Arecibo,81966,10596,72013,50079.56
7,Arroyo,9.704011e+06,Arroyo,17238,8611,72015,56294.29
8,Barceloneta,1.259085e+07,Barceloneta,23727,9847,72017,53065.48
9,Barranquitas,2.161582e+07,Barranquitas,27725,8537,72019,77965.10


In [119]:
# Extract the aggregated registrations per county
registrationsPerCountyDf = femaDf['county'].value_counts().reset_index()
#registrationsPerCountyDf = femaDf[femaDf['homeDamage'] == True]['county'].value_counts().reset_index()

registrationsPerCountyDf.rename(columns={'index': 'county', 'county': 'registrations'}, inplace=True)

# Drop the Statewide row
registrationsPerCountyDf = registrationsPerCountyDf[registrationsPerCountyDf['county'] != 'Statewide']

# Format the county column
registrationsPerCountyDf['county'] = registrationsPerCountyDf.county.str.extract('(.+?) \(')

registrationsPerCountyDf = pd.merge(registrationsPerCountyDf, 
                                   countyDemographicsDf[['county_name','pop_estimates_2019', 'fips_code']],
                                   left_on='county', right_on='county_name', how='inner')

# Calculate the ihpAmount per capita
registrationsPerCountyDf['registrationsPerCapita'] = np.round(registrationsPerCountyDf['registrations']/registrationsPerCountyDf['pop_estimates_2019']*100,decimals=2)

registrationsPerCountyDf.head(20)

,county,registrations,county_name,pop_estimates_2019,fips_code,registrationsPerCapita
0,San Juan,122594,San Juan,318441,72127,38.50
1,Bayamon,63583,Bayamon,169269,72021,37.56
2,Carolina,52319,Carolina,146984,72031,35.60
3,Ponce,50148,Ponce,131881,72113,38.03
4,Caguas,42829,Caguas,124606,72025,34.37
5,Arecibo,30047,Arecibo,81966,72013,36.66
6,Guaynabo,28141,Guaynabo,83728,72061,33.61
7,Toa Baja,27799,Toa Baja,74271,72137,37.43
8,Mayaguez,24827,Mayaguez,71530,72097,34.71
9,Toa Alta,21041,Toa Alta,72025,72135,29.21


### Choropleth Map using plotly (https://plotly.com/python/county-choropleth/#single-state)

In [20]:
# Install plotly dependencies
#!pip install plotly
#!pip install plotly-geo
#!pip install geopandas==0.3.0
#!pip install pyshp==1.2.10
#!pip install shapely==1.6.3
import plotly.figure_factory as ff

In [23]:
# Per Capita Income (2019) by county
values = ihpAndIncomePerCapitaDf['per_capita_income_past_year'].tolist()
fips = ihpAndIncomePerCapitaDf['fips_code'].tolist()

endpts = list(np.mgrid[min(values):max(values):5j])
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]
incomePerCapitaFig = ff.create_choropleth(
    fips=fips, values=values, scope=['PR'], show_state_data=True,
    colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Income Per Capita by County',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.25},    
    exponent_format=True,
)
incomePerCapitaFig.layout.template = None
incomePerCapitaFig.show()

In [54]:
# ihp Amount Per Capita by County
values = ihpAndIncomePerCapitaDf['ihpAmountPerCapita'].tolist()
fips = ihpAndIncomePerCapitaDf['fips_code'].tolist()

endpts = list(np.mgrid[min(values):max(values):5j])
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]
ihpPerCapitaFig = ff.create_choropleth(
    fips=fips, values=values, scope=['PR'], show_state_data=True,
    colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='IHP Amount Awarded Per Capita by County',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.25},
    exponent_format=True,
)
ihpPerCapitaFig.layout.template = None
ihpPerCapitaFig.show()

In [120]:
# Registrations Per Capita by County
values = registrationsPerCountyDf['registrationsPerCapita'].tolist()
fips = registrationsPerCountyDf['fips_code'].tolist()

endpts = list(np.mgrid[min(values):max(values):5j])
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]
registrationsPerCapitaFig = ff.create_choropleth(
    fips=fips, values=values, scope=['PR'], show_state_data=True,
    colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Applications Per Capita by County',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.25},
    exponent_format=True,
)
registrationsPerCapitaFig.layout.template = None
registrationsPerCapitaFig.show()

### haStatus codes for denied applications

In [328]:
from collections import defaultdict
haStatusCodes = defaultdict(int)

statusCodes = femaDf[(femaDf['ihpAmount'] == 0) & (femaDf['haStatus'] != 'TSA - Transitional Sheltering Assistance') & (femaDf.haStatus.notnull())]['haStatus'].str.extract('(.+?)\s?-')[0].str.split(',')
#statusCodes = femaDf[(femaDf.haStatus.notnull())]['haStatus'].str.extract('(.+?)\s?-')[0].str.split(',')
statusCodes = statusCodes.dropna()
for index, value in statusCodes.items(): 
    for statusCode in value:
        haStatusCodes[statusCode] = haStatusCodes.get(statusCode, 0) + 1
        
haStatusCodesDf = pd.DataFrame(list(haStatusCodes.items()), columns=['statusCode', 'Count'])

In [329]:
import plotly.express as px

fig = px.pie(haStatusCodesDf, values='Count', names='statusCode', 
             color_discrete_sequence=px.colors.sequential.tempo)
fig.update_traces(hoverinfo='label+percent+name',textposition='inside', textinfo='label+percent')
fig.update(layout_title_text='Housing Assistance Status Codes for Denied Applications',
           layout_showlegend=False)
fig.show()